# ChunkFlow: REST API Usage

This notebook demonstrates how to use ChunkFlow's REST API for integrating chunking capabilities into your applications.

## What You'll Learn

1. How to start the ChunkFlow API server
2. How to use all API endpoints
3. How to integrate ChunkFlow into your applications
4. Best practices for API usage

## Prerequisites

```bash
pip install chunk-flow[api,huggingface]
```

## Starting the API Server

Before running this notebook, start the API server in a separate terminal:

```bash
# Start the server
uvicorn chunk_flow.api.app:app --reload --port 8000

# Or with Docker
docker-compose up
```

The API will be available at `http://localhost:8000`

**API Documentation**: Once started, visit:
- Swagger UI: http://localhost:8000/docs
- ReDoc: http://localhost:8000/redoc

In [ ]:
# Import required libraries
import requests
import json
from pprint import pprint

# API base URL
API_BASE_URL = "http://localhost:8000"

print("✓ Imports successful!")
print(f"API Base URL: {API_BASE_URL}")

## 1. Health Check

First, verify the API is running.

In [ ]:
# Check API health
response = requests.get(f"{API_BASE_URL}/health")

if response.status_code == 200:
    print("✓ API is healthy!\n")
    pprint(response.json())
else:
    print(f"✗ API health check failed: {response.status_code}")
    print("Make sure the API server is running: uvicorn chunk_flow.api.app:app --reload")

## 2. Discover Available Strategies

Get list of available chunking strategies.

In [ ]:
# Get available strategies
response = requests.get(f"{API_BASE_URL}/strategies")

if response.status_code == 200:
    strategies = response.json()
    print("Available Chunking Strategies:\n")
    for strategy in strategies["strategies"]:
        print(f"  • {strategy}")
else:
    print(f"Error: {response.status_code}")

## 3. Discover Available Metrics

Get list of available evaluation metrics.

In [ ]:
# Get available metrics
response = requests.get(f"{API_BASE_URL}/metrics")

if response.status_code == 200:
    metrics = response.json()
    print("Available Evaluation Metrics:\n")
    for metric in metrics["metrics"]:
        print(f"  • {metric}")
else:
    print(f"Error: {response.status_code}")

## 4. Discover Available Embedding Providers

Get list of available embedding providers.

In [ ]:
# Get available providers
response = requests.get(f"{API_BASE_URL}/providers")

if response.status_code == 200:
    providers = response.json()
    print("Available Embedding Providers:\n")
    for provider in providers["providers"]:
        print(f"  • {provider}")
else:
    print(f"Error: {response.status_code}")

## 5. Chunk Text

The core API endpoint - chunk text with a specified strategy.

In [ ]:
# Sample document
document = """
# Python Programming

Python is a high-level, interpreted programming language known for its simplicity and readability. 
Created by Guido van Rossum and first released in 1991, Python emphasizes code readability with 
its notable use of significant whitespace.

## Key Features

Python supports multiple programming paradigms, including procedural, object-oriented, and functional 
programming. It has a comprehensive standard library and a vast ecosystem of third-party packages.

## Popular Applications

Python is widely used in web development, data science, machine learning, automation, and scientific 
computing. Its simplicity makes it an excellent choice for beginners while remaining powerful enough 
for complex applications.
"""

# Prepare request
chunk_request = {
    "text": document,
    "strategy": "recursive",
    "strategy_config": {
        "chunk_size": 300,
        "overlap": 50
    },
    "doc_id": "python_intro"
}

# Make request
response = requests.post(
    f"{API_BASE_URL}/chunk",
    json=chunk_request
)

if response.status_code == 200:
    result = response.json()
    print(f"✓ Chunking successful!\n")
    print(f"Number of chunks: {result['num_chunks']}")
    print(f"Processing time: {result['processing_time_ms']:.2f}ms\n")
    
    print("Chunks:")
    for i, chunk in enumerate(result['chunks'][:3], 1):  # Show first 3
        print(f"\n{i}. {chunk[:150]}...")
    
    # Save chunks for later use
    chunks = result['chunks']
else:
    print(f"✗ Error: {response.status_code}")
    print(response.json())

## 6. Generate Embeddings

Generate embeddings for chunks using an embedding provider.

In [ ]:
# Prepare embedding request
embed_request = {
    "texts": chunks,  # Use chunks from previous step
    "provider": "huggingface",
    "provider_config": {
        "model": "sentence-transformers/all-MiniLM-L6-v2",
        "normalize": True
    }
}

# Make request
response = requests.post(
    f"{API_BASE_URL}/embed",
    json=embed_request
)

if response.status_code == 200:
    result = response.json()
    print(f"✓ Embedding generation successful!\n")
    print(f"Number of embeddings: {result['num_embeddings']}")
    print(f"Embedding dimensions: {result['dimensions']}")
    print(f"Processing time: {result['processing_time_ms']:.2f}ms")
    print(f"Total tokens: {result['token_count']}")
    
    # Save embeddings for later use
    embeddings = result['embeddings']
else:
    print(f"✗ Error: {response.status_code}")
    print(response.json())

## 7. Evaluate Chunks

Evaluate chunking quality using metrics.

In [ ]:
# Prepare evaluation request
eval_request = {
    "chunks": chunks,
    "embeddings": embeddings,
    "metrics": [
        "semantic_coherence",
        "boundary_quality",
        "chunk_stickiness",
        "topic_diversity"
    ]
}

# Make request
response = requests.post(
    f"{API_BASE_URL}/evaluate",
    json=eval_request
)

if response.status_code == 200:
    result = response.json()
    print(f"✓ Evaluation successful!\n")
    print("Metric Results:\n")
    
    for metric_name, metric_data in result['metrics'].items():
        score = metric_data['score']
        print(f"  {metric_name:25s} {score:.4f}")
else:
    print(f"✗ Error: {response.status_code}")
    print(response.json())

## 8. Compare Strategies

Compare multiple strategies on the same document.

In [ ]:
# Prepare comparison request
compare_request = {
    "text": document,
    "strategies": [
        {
            "name": "fixed_size",
            "config": {"chunk_size": 300, "overlap": 50}
        },
        {
            "name": "recursive",
            "config": {"chunk_size": 400, "overlap": 60}
        },
        {
            "name": "markdown",
            "config": {"respect_headers": True}
        }
    ],
    "embedding_provider": "huggingface",
    "embedding_config": {
        "model": "sentence-transformers/all-MiniLM-L6-v2"
    },
    "metrics": [
        "semantic_coherence",
        "boundary_quality"
    ]
}

# Make request
response = requests.post(
    f"{API_BASE_URL}/compare",
    json=compare_request
)

if response.status_code == 200:
    result = response.json()
    print(f"✓ Strategy comparison successful!\n")
    
    print("Comparison Results:\n")
    print(f"{'Strategy':<20} {'Chunks':<10} {'Coherence':<12} {'Boundary':<12}")
    print("-" * 60)
    
    for strategy_name, strategy_data in result['strategies'].items():
        num_chunks = strategy_data['num_chunks']
        coherence = strategy_data['metric_results'].get('semantic_coherence', {}).get('score', 0)
        boundary = strategy_data['metric_results'].get('boundary_quality', {}).get('score', 0)
        
        print(f"{strategy_name:<20} {num_chunks:<10} {coherence:<12.4f} {boundary:<12.4f}")
    
    print(f"\nBest Strategy: {result['best_strategy']}")
else:
    print(f"✗ Error: {response.status_code}")
    print(response.json())

## 9. Complete Workflow Example

A complete end-to-end workflow using the API.

In [ ]:
def chunk_and_evaluate_via_api(text, strategy="recursive", chunk_size=500, overlap=80):
    """
    Complete chunking and evaluation workflow via API.
    
    Args:
        text: Document to chunk
        strategy: Chunking strategy to use
        chunk_size: Size of chunks
        overlap: Overlap between chunks
    
    Returns:
        dict with chunks, embeddings, and evaluation results
    """
    # Step 1: Chunk the document
    chunk_response = requests.post(
        f"{API_BASE_URL}/chunk",
        json={
            "text": text,
            "strategy": strategy,
            "strategy_config": {"chunk_size": chunk_size, "overlap": overlap}
        }
    )
    
    if chunk_response.status_code != 200:
        raise Exception(f"Chunking failed: {chunk_response.json()}")
    
    chunks = chunk_response.json()["chunks"]
    print(f"✓ Created {len(chunks)} chunks")
    
    # Step 2: Generate embeddings
    embed_response = requests.post(
        f"{API_BASE_URL}/embed",
        json={
            "texts": chunks,
            "provider": "huggingface",
            "provider_config": {"model": "sentence-transformers/all-MiniLM-L6-v2"}
        }
    )
    
    if embed_response.status_code != 200:
        raise Exception(f"Embedding failed: {embed_response.json()}")
    
    embeddings = embed_response.json()["embeddings"]
    print(f"✓ Generated {len(embeddings)} embeddings")
    
    # Step 3: Evaluate quality
    eval_response = requests.post(
        f"{API_BASE_URL}/evaluate",
        json={
            "chunks": chunks,
            "embeddings": embeddings,
            "metrics": ["semantic_coherence", "boundary_quality"]
        }
    )
    
    if eval_response.status_code != 200:
        raise Exception(f"Evaluation failed: {eval_response.json()}")
    
    metrics = eval_response.json()["metrics"]
    print(f"✓ Evaluation complete")
    
    return {
        "chunks": chunks,
        "embeddings": embeddings,
        "metrics": metrics
    }

# Test the workflow
print("Running complete workflow...\n")

result = chunk_and_evaluate_via_api(
    text=document,
    strategy="recursive",
    chunk_size=400,
    overlap=60
)

print("\nResults:")
print(f"  Chunks: {len(result['chunks'])}")
print(f"  Embeddings: {len(result['embeddings'])}")
print(f"\nMetrics:")
for metric_name, metric_data in result['metrics'].items():
    print(f"  {metric_name}: {metric_data['score']:.4f}")

## 10. Error Handling

Best practices for handling API errors.

In [ ]:
def safe_api_call(endpoint, method="GET", json_data=None):
    """
    Make an API call with proper error handling.
    
    Args:
        endpoint: API endpoint (e.g., "/chunk")
        method: HTTP method (GET or POST)
        json_data: Request body for POST requests
    
    Returns:
        Response JSON or None if error
    """
    url = f"{API_BASE_URL}{endpoint}"
    
    try:
        if method == "GET":
            response = requests.get(url, timeout=30)
        elif method == "POST":
            response = requests.post(url, json=json_data, timeout=30)
        else:
            raise ValueError(f"Unsupported method: {method}")
        
        # Check for HTTP errors
        response.raise_for_status()
        
        return response.json()
        
    except requests.exceptions.ConnectionError:
        print(f"✗ Error: Could not connect to API at {url}")
        print("  Make sure the API server is running")
        return None
        
    except requests.exceptions.Timeout:
        print(f"✗ Error: Request timed out")
        return None
        
    except requests.exceptions.HTTPError as e:
        print(f"✗ HTTP Error: {e.response.status_code}")
        try:
            error_detail = e.response.json()
            print(f"  Detail: {error_detail.get('detail', 'No details available')}")
        except:
            print(f"  Response: {e.response.text}")
        return None
        
    except Exception as e:
        print(f"✗ Unexpected error: {e}")
        return None

# Test error handling
print("Testing error handling:\n")

# Valid request
result = safe_api_call("/health", method="GET")
if result:
    print(f"✓ Health check successful: {result['status']}")

# Invalid strategy (will cause error)
print("\nTesting invalid strategy:")
result = safe_api_call(
    "/chunk",
    method="POST",
    json_data={
        "text": "Sample text",
        "strategy": "invalid_strategy_name",
        "strategy_config": {}
    }
)

## 11. Integration Example: Python Application

Example of integrating ChunkFlow API into a Python application.

In [ ]:
class ChunkFlowClient:
    """
    Client wrapper for ChunkFlow API.
    """
    
    def __init__(self, base_url="http://localhost:8000"):
        self.base_url = base_url
        self.session = requests.Session()
    
    def chunk(self, text, strategy="recursive", **config):
        """Chunk text using specified strategy."""
        response = self.session.post(
            f"{self.base_url}/chunk",
            json={"text": text, "strategy": strategy, "strategy_config": config}
        )
        response.raise_for_status()
        return response.json()["chunks"]
    
    def embed(self, texts, provider="huggingface", **config):
        """Generate embeddings for texts."""
        response = self.session.post(
            f"{self.base_url}/embed",
            json={"texts": texts, "provider": provider, "provider_config": config}
        )
        response.raise_for_status()
        return response.json()["embeddings"]
    
    def evaluate(self, chunks, embeddings, metrics):
        """Evaluate chunk quality."""
        response = self.session.post(
            f"{self.base_url}/evaluate",
            json={"chunks": chunks, "embeddings": embeddings, "metrics": metrics}
        )
        response.raise_for_status()
        return response.json()["metrics"]
    
    def compare_strategies(self, text, strategies, metrics, embedding_provider="huggingface", **embed_config):
        """Compare multiple strategies."""
        response = self.session.post(
            f"{self.base_url}/compare",
            json={
                "text": text,
                "strategies": strategies,
                "embedding_provider": embedding_provider,
                "embedding_config": embed_config,
                "metrics": metrics
            }
        )
        response.raise_for_status()
        return response.json()
    
    def health(self):
        """Check API health."""
        response = self.session.get(f"{self.base_url}/health")
        response.raise_for_status()
        return response.json()

# Use the client
client = ChunkFlowClient()

# Check health
print("Health check:")
print(client.health())

# Chunk text
print("\nChunking text...")
chunks = client.chunk(document, strategy="recursive", chunk_size=400, overlap=60)
print(f"Created {len(chunks)} chunks")

# Generate embeddings
print("\nGenerating embeddings...")
embeddings = client.embed(
    chunks,
    provider="huggingface",
    model="sentence-transformers/all-MiniLM-L6-v2"
)
print(f"Generated {len(embeddings)} embeddings")

# Evaluate
print("\nEvaluating...")
metrics = client.evaluate(
    chunks,
    embeddings,
    metrics=["semantic_coherence", "boundary_quality"]
)
for metric_name, metric_data in metrics.items():
    print(f"  {metric_name}: {metric_data['score']:.4f}")

## 12. Best Practices

### API Usage Tips

1. **Connection Pooling**: Use `requests.Session()` for multiple requests
2. **Error Handling**: Always handle connection errors, timeouts, and HTTP errors
3. **Timeouts**: Set appropriate timeouts for long-running operations
4. **Retries**: Implement retry logic for transient failures
5. **Rate Limiting**: Respect API rate limits if deployed

### Performance Tips

1. **Batch Processing**: Process multiple chunks at once when possible
2. **Caching**: Cache embeddings for repeated use
3. **Async**: Use async clients (httpx) for concurrent requests
4. **Compression**: Enable gzip compression for large payloads

### Security Tips

1. **HTTPS**: Always use HTTPS in production
2. **Authentication**: Implement API key authentication
3. **Input Validation**: Validate inputs before sending to API
4. **Secrets**: Never hardcode API keys in code

## 13. Docker Deployment

### Running with Docker

```bash
# Build image
docker build -t chunkflow:latest .

# Run container
docker run -p 8000:8000 chunkflow:latest

# Or use docker-compose
docker-compose up
```

### Environment Variables

```bash
# .env file
OPENAI_API_KEY=your-key-here
LOG_LEVEL=INFO
MAX_CHUNK_SIZE=10000
```

### Health Monitoring

```python
# Check health periodically
import time

def monitor_api_health(interval=60):
    while True:
        try:
            response = requests.get(f"{API_BASE_URL}/health", timeout=5)
            if response.status_code == 200:
                print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] API is healthy")
            else:
                print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] API unhealthy: {response.status_code}")
        except Exception as e:
            print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] API check failed: {e}")
        
        time.sleep(interval)

# monitor_api_health()  # Uncomment to run
```

## Summary

In this notebook, you learned:

✅ How to use all ChunkFlow API endpoints
✅ How to chunk text, generate embeddings, and evaluate quality via API
✅ How to compare multiple strategies
✅ How to handle errors properly
✅ How to create a reusable API client
✅ Best practices for API usage
✅ Docker deployment tips

## API Endpoints Summary

| Endpoint | Method | Purpose |
|----------|--------|----------|
| `/` | GET | Welcome message |
| `/health` | GET | Health check |
| `/strategies` | GET | List available strategies |
| `/metrics` | GET | List available metrics |
| `/providers` | GET | List available providers |
| `/chunk` | POST | Chunk text |
| `/embed` | POST | Generate embeddings |
| `/evaluate` | POST | Evaluate chunks |
| `/compare` | POST | Compare strategies |

## Next Steps

- Deploy the API to production (AWS, GCP, Azure)
- Implement authentication and rate limiting
- Set up monitoring and logging
- Build a web UI on top of the API
- Integrate into your RAG pipeline

## Resources

- **API Docs**: http://localhost:8000/docs
- **ReDoc**: http://localhost:8000/redoc
- **Docker Guide**: See DOCKER.md in repo
- **GitHub**: https://github.com/chunkflow/chunk-flow